In [1]:
!pip install modin[all]

In [2]:
import modin.pandas as pd
import pandas

#############################################
### For the purpose of timing comparisons ###
#############################################
import time
import ray
ray.init()
#############################################


2023-01-04 14:16:34,753	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.12
Ray version:,2.0.1
Dashboard:,http://127.0.0.1:8265


In [3]:
#Modin is able to utilize Ray’s built-in autoscaled cluster. To launch a Ray cluster using Amazon Web Service (AWS), you can use this file as the config file.

In [4]:
pip install boto3 aws configure

In [5]:
import pandas
import numpy as np
df = pandas.concat([pandas.DataFrame(np.random.randint(0, 100, size=(2**20, 2**8))) for _ in range(40)]) # Memory Error!

MemoryError: Unable to allocate 1.00 GiB for an array with shape (1048576, 256) and data type int32

When we run this on a laptop with 32GB of RAM, pandas will run out of memory and throw an error (e.g., MemoryError , Killed: 9).

pandas makes use of in-memory data structures to store and operate on data, which means that if you have a dataset that is too large to fit in memory, it will cause an error on pandas. As an example, let’s creates a 80GB DataFrame by appending together 40 different 2GB DataFrames.

Operating on out-of-memory data with Modin
In order to work with data that exceeds memory constraints, you can use Modin to handle these large datasets.

In [6]:
import modin.pandas as pd
import numpy as np
df = pd.concat([pd.DataFrame(np.random.randint(0, 100, size=(2**20, 2**8))) for _ in range(40)]) # 40x2GB frames -- Working!
df.info()

(pid=) [2023-01-04 14:38:50,949 C 19276 14816] (raylet.exe) dlmalloc.cc:129:  Check failed: *handle != nullptr CreateFileMapping() failed. GetLastError() = 1455
(pid=) *** StackTrace Information ***
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) unknown
(pid=) configthreadlocale
(pid=) BaseThreadInitThunk
(pid=) RtlUserThreadStart
(pid=) 


LocalRayletDiedError: The task's local raylet died. Check raylet.out for more information.

2023-01-04 14:39:20,296	WARNING worker.py:1829 -- The node with node id: 784e56fdb1333c91a8869df5507179eab49a3e1e25dbf7bb282a44a6 and address: 127.0.0.1 and node name: 127.0.0.1 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, preempted node, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.


In [7]:
import modin
modin.config.Engine.put("Dask")

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()



In [ ]:
import modin.pandas as pd
import numpy as np
df = pd.concat([pd.DataFrame(np.random.randint(0, 100, size=(2**20, 2**8))) for _ in range(40)]) # 40x2GB frames -- Working!
df.info()

distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\distributed\protocol\core.py", line 76, in dumps
    frames[0] = msgpack.dumps(msg, default=_encode_default, use_bin_type=True)
  File "C:\Users\user\anaconda3\lib\site-packages\msgpack\__init__.py", line 35, in packb
    return Packer(**kwargs).pack(o)
  File "msgpack/_packer.pyx", line 120, in msgpack._cmsgpack.Packer.__cinit__
MemoryError: Unable to allocate internal buffer.
distributed.comm.utils - ERROR - Unable to allocate internal buffer.
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\distributed\comm\utils.py", line 53, in _to_frames
    return list(protocol.dumps(msg, **kwargs))
  File "C:\Users\user\anaconda3\lib\site-packages\distributed\protocol\core.py", line 76, in dumps
    frames[0] = msgpack.dumps(msg, default=_encode_default, use_bin_type=True)
  File "C:\Users\user\anaconda3\lib\site-pack

In [ ]:
2**20

In [ ]:
2**8